## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("D:/GW_Data_Analytic_Boot_Camp/Module6/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,66.00,62,31,13.00,scattered clouds
1,1,Lagoa,PT,39.0500,-27.9833,60.01,87,95,30.89,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,48.00,23,90,14.97,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,20,9.22,light rain
4,4,Urozhaynoye,RU,44.7883,44.9228,48.34,74,92,7.72,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Puerto Ayora,EC,-0.7393,-90.3518,82.40,74,20,9.22,light rain
6,6,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,79.23,83,17,8.34,few clouds
7,7,Sibolga,ID,1.7427,98.7792,75.24,94,100,9.06,overcast clouds
11,11,Lorengau,PG,-2.0226,147.2712,80.80,78,100,7.05,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,75.87,70,37,14.90,light rain
19,19,Pyapon,MM,16.2833,95.6833,75.72,82,48,5.30,scattered clouds
21,21,Ahuimanu,US,21.4447,-157.8378,80.60,69,75,17.27,light rain
29,29,Alto Araguaia,BR,-17.3147,-53.2153,76.86,65,88,2.75,overcast clouds
33,33,Hobyo,SO,5.3505,48.5268,82.11,73,6,13.35,clear sky
35,35,Victoria,HK,22.2855,114.1577,80.01,81,9,3.13,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,82.40,light rain,-0.7393,-90.3518,
6,Sao Jose Da Coroa Grande,BR,79.23,few clouds,-8.8978,-35.1478,
7,Sibolga,ID,75.24,overcast clouds,1.7427,98.7792,
11,Lorengau,PG,80.80,overcast clouds,-2.0226,147.2712,
17,Rikitea,PF,75.87,light rain,-23.1203,-134.9692,
19,Pyapon,MM,75.72,scattered clouds,16.2833,95.6833,
21,Ahuimanu,US,80.60,light rain,21.4447,-157.8378,
29,Alto Araguaia,BR,76.86,overcast clouds,-17.3147,-53.2153,
33,Hobyo,SO,82.11,clear sky,5.3505,48.5268,
35,Victoria,HK,80.01,clear sky,22.2855,114.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Check number of records with missing Hotel Name before drop
hotel_df.loc[hotel_df["Hotel Name"] == ""].count()["Hotel Name"]

14

In [10]:
hotel_df.count()

City                   224
Country                224
Max Temp               224
Current Description    224
Lat                    224
Lng                    224
Hotel Name             224
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

In [12]:
# Check number of records after remove rows with missing values.
clean_hotel_df.count()

City                   210
Country                210
Max Temp               210
Current Description    210
Lat                    210
Lng                    210
Hotel Name             210
dtype: int64

In [13]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Puerto Ayora,EC,82.40,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Sao Jose Da Coroa Grande,BR,79.23,few clouds,-8.8978,-35.1478,Crôa Mares Hotel
7,Sibolga,ID,75.24,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
11,Lorengau,PG,80.80,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
17,Rikitea,PF,75.87,light rain,-23.1203,-134.9692,Pension Maro'i
19,Pyapon,MM,75.72,scattered clouds,16.2833,95.6833,Aung Naing Thu Motel
21,Ahuimanu,US,80.60,light rain,21.4447,-157.8378,Paradise Bay Resort
29,Alto Araguaia,BR,76.86,overcast clouds,-17.3147,-53.2153,Araguaia Palace Hotel
33,Hobyo,SO,82.11,clear sky,5.3505,48.5268,Halane Hotel and Restaurent
35,Victoria,HK,80.01,clear sky,22.2855,114.1577,Mini Hotel Central


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><strong>Hotel Name</strong></dt>
<dt>{Hotel Name}</dt>
<dt><strong>City</strong></dt>
<dt>{City}</dt>
<dt><strong>Country</strong></dt>
<dt>{Country}</dt>
<dt><strong>Current Weather</strong></dt>
<dt>{Current Description} and {Max Temp} °F</dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))